In [26]:
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, MaxoutDense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge,Input,merge,UpSampling2D,BatchNormalization

from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
import numpy as np

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import defaultdict
import PIL.Image

import json
from tqdm import tqdm

from keras.optimizers import SGD, RMSprop, Adam

from utils import *
import cPickle as pickle
from matplotlib import pyplot as plt

from itertools import compress

import shutil
import string

import collections

import matplotlib.patches as patches

import os

from keras import callbacks

In [2]:
def get_unet(img_rows, img_cols,nr_labels):
    
    
    inputs = Input((3, img_rows, img_cols))
    bn = BatchNormalization()(inputs)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(bn)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(nr_labels, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def normalize_imgs(data):
    
    data = data.astype('float32')

    mean = np.mean(data)  # mean for data centering
    std = np.std(data)  # std for data normalization

    data -= mean
    data /= std
    
    return data

def scale_masks(data):
    data = data.astype('float32')
    data /= 255.  # scale masks to [0, 1]
    
    return data
    
def plotPredictedVsReality(predictions, images, masks, min_index, max_index):
    
    nr_rows = max_index - min_index
    nr_cols = 2
    pic_index = 1
    
    plt.figure(figsize=(15,15))
    for index in range(min_index,max_index):
   
        raw_pred = predictions[index]
        squeezed_pred = np.squeeze(raw_pred)
#         squeezed_pred = threshold(squeezed_pred)

        raw_mask = masks[index]
        raw_mask = np.squeeze(raw_mask)
        
        image = images[index]
        image = np.transpose(image,(1,2,0))
        
        nr_cols = 3
        plt.subplot(1,nr_cols,1)
        pic_index+=1
        plt.imshow(squeezed_pred, cmap='Greys',  interpolation='nearest')
        
        plt.subplot(1,nr_cols,2)
        plt.imshow(raw_mask)
        pic_index+=1
        
        plt.subplot(1,nr_cols,3)
        plt.imshow(image)
        pic_index+=1
        
        plt.figure(figsize=(15,15))
        
    plt.show()

In [3]:
base_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/detection/"

In [4]:
categories = ['vehicle','person']

In [5]:
def get_association_arr_for_category(category):
    full_path = base_path+category+"-category/train2014/"
    prev_paths =  os.listdir(full_path)
    prev_paths.sort()
    
    nr_pics = len(prev_paths) / 3
    print("Category %s => %d pics" % (category,nr_pics))

    pic_2_mask_2_prev_paths = [(prev_paths[i],prev_paths[i+1],prev_paths[i+2]) for i in range(0,nr_pics*3,3)]
    
    picName_2_data_dict = {}
    
    for (pic_path,mask_path,prev_path) in tqdm(pic_2_mask_2_prev_paths[:1000]):
        
        pic = PIL.Image.open(full_path+pic_path).resize((224, 224), PIL.Image.NEAREST)
        pic = np.asarray(pic)
        
        prev = PIL.Image.open(full_path+prev_path).resize((224, 224), PIL.Image.NEAREST)
        prev = np.asarray(prev)

        mask = PIL.Image.open(full_path+mask_path).resize((224, 224), PIL.Image.NEAREST)
        mask = np.asarray(mask)
    
        picName_2_data_dict[pic_path] = (pic,prev,mask)
     
    return picName_2_data_dict

In [6]:
cat_2_assocArr_dict = dict([(category,get_association_arr_for_category(category)) for category in categories])

  1%|          | 11/1000 [00:00<00:09, 109.13it/s]

Category vehicle => 19215 pics


  1%|          | 8/1000 [00:00<00:12, 76.36it/s]

Category person => 45174 pics


100%|██████████| 1000/1000 [00:09<00:00, 104.11it/s]


In [9]:
for cat in cat_2_assocArr_dict:
    print("%s => %d pictures" % (cat,len(cat_2_assocArr_dict[cat])))

person => 1000 pictures
vehicle => 1000 pictures


In [142]:
# nr_pics = 3

# for cat in cat_2_assocArr_dict:
#     outdoor_assoc_arr =  cat_2_assocArr_dict[cat]

#     i = 0
#     plt.figure(figsize=(15,15))
#     for picName, (pic,prev,mask) in outdoor_assoc_arr.iteritems():
#         nr_cols = 3
#         plt.subplot(1,nr_cols,1)
#         plt.imshow(pic)
        
#         plt.subplot(1,nr_cols,2)
#         plt.imshow(prev)
        
#         plt.subplot(1,nr_cols,3)
#         plt.imshow(mask)
        
#         plt.figure(figsize=(15,15))
#         i+=1
#         if(i == nr_pics):
#             break

# plt.show()

In [20]:
class RawPicData(object):

    def __init__(self,category,pic,prev,mask):
        self.category_2_data_dict = {}
        self.add_category(category,pic,prev,mask)
        
    def add_category(self,category,pic,prev,mask):
        self.category_2_data_dict[category] = (pic,prev,mask)

class PicData(object):

    def __init__(self,pic):
        self.category_2_prevMask_dict = {}
        self.pic = pic
        
    def add_category(self,category,prev,mask):
        self.category_2_prevMask_dict[category] = (prev,mask)

In [21]:
picName_2_rawPicData_dict = {}

for catName, picDataDict in cat_2_assocArr_dict.iteritems():

    for picName,(pic,prev,mask) in picDataDict.iteritems():
        
        if(picName in picName_2_rawPicData_dict):
            rawPicData = picName_2_rawPicData_dict[picName]
            rawPicData.add_category(catName,pic,prev,mask)
        else:
            picName_2_rawPicData_dict[picName] = RawPicData(catName,pic,prev,mask)

In [23]:
print("Unique pictures => %d"%len(picName_2_rawPicData_dict))

Unique pictures => 1708


In [28]:
count_dict = defaultdict(int)

for picName, rawPicData in picName_2_rawPicData_dict.iteritems():
    for catName in rawPicData.category_2_data_dict:
        current_value = count_dict[catName]
        current_value += 1
        count_dict[catName] = current_value
    
        
for catName,count in count_dict.iteritems():
    print("%s => %d pictures"%(catName,count))


person => 1000 pictures
vehicle => 1000 pictures


# Convert rawPicData to PicData

In [30]:
picName_2_picData_dict = {}

for picName, rawPicData in picName_2_rawPicData_dict.iteritems():
    category_2_data_dict = rawPicData.category_2_data_dict
    #doesn't matter the category, all pics are the same
    (pic,_,_) = category_2_data_dict[category_2_data_dict.keys()[0]]
    picData = PicData(pic)
    for category, (pic,prev,mask) in category_2_data_dict.iteritems():
        picData.add_category(category,prev,mask)

    picName_2_picData_dict[picName] = picData

In [31]:
len(picName_2_picData_dict)

1708

In [32]:
count_dict = defaultdict(int)

for picName, picData in picName_2_picData_dict.iteritems(): 
     for catName in picData.category_2_prevMask_dict:
        current_value = count_dict[catName]
        current_value += 1
        count_dict[catName] = current_value

for catName,count in count_dict.iteritems():
    print("%s => %d pictures"%(catName,count))

person => 1000 pictures
vehicle => 1000 pictures


In [35]:
cat_2_masks = defaultdict(list)
cat_2_prev = defaultdict(list)

for picName, picData in picName_2_picData_dict.iteritems():
    
    for category in categories:
        mask_to_append = None
        prev_to_append = None
        
        if(category in picData.category_2_prevMask_dict):
            (prev,mask) = picData.category_2_prevMask_dict[category]
            mask_to_append = mask
            prev_to_append = prev
            
        else:
            default_mask=np.empty((224,224))
            default_mask.fill(0)
            mask_to_append = default_mask

            default_prev=np.empty((224,224,3))
            default_prev.fill(0)
            prev_to_append = default_prev
    
        current_masks = cat_2_masks[category]
        current_prev = cat_2_prev[category]
        
        current_masks.append(mask_to_append)
        current_prev.append(prev_to_append)
        
        cat_2_masks[category] = current_masks
        cat_2_prev[category] = current_prev

In [39]:
for cat,masks in cat_2_masks.iteritems():
    print("%s => %d length"%(cat,len(masks)))
    
for cat,prev in cat_2_prev.iteritems():
    print("%s => %d length"%(cat,len(prev)))

person => 1708 length
vehicle => 1708 length
person => 1708 length
vehicle => 1708 length


In [141]:
# for cat in categories:
#     masks = cat_2_masks[cat]
#     prevs = cat_2_prev[cat]
    
#     nr_pics = 3
#     for i in range(nr_pics):
#         nr_cols = 2
#         plt.subplot(1,nr_cols,1)
#         plt.imshow(prevs[i])

#         plt.subplot(1,nr_cols,2)
#         plt.imshow(masks[i])

#         plt.figure(figsize=(15,15))
    
#     plt.show()

In [58]:
pics = [picData.pic for picData in picName_2_picData_dict.values()]
pics_stacked = np.stack(pics,axis=0)
pics_transposed = np.transpose(pics_stacked,(0,3,1,2))

In [70]:
cat_2_index_dict = dict([(cat,index) for index,cat in enumerate(cat_2_masks)])
cat_2_index_dict

{'person': 0, 'vehicle': 1}

In [59]:
all_masks = [masks for _,masks in cat_2_masks.iteritems()]
masks_concat = np.stack(all_masks,axis=1)

In [65]:
train_pics =  pics_transposed
train_masks = masks_concat

print(train_pics.shape)
print(train_masks.shape)

(1708, 3, 224, 224)
(1708, 2, 224, 224)


In [62]:
nr_labels = len(categories)
model = get_unet(224,224,nr_labels)

In [111]:
model.fit(train_pics, 
          train_masks, 
          batch_size=32, 
          nb_epoch=5,
         )

Epoch 1/5
1708/1708 [==============================] - 76s - loss: 351.3690 - acc: 0.0525     

In [ ]:
# save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/detection/food2-dataset-4cls/models/"
# model.save_weights(save_path+"unet_3e_bn.h5")

In [112]:
no_predictions = 30
predictions = model.predict(train_pics[:no_predictions])
predictions.shape

(30, 2, 224, 224)

In [113]:
cat_2_predictions_dict = {}
for cat,index in cat_2_index_dict.iteritems():
    print("%s => %d"%(cat,index))
    cat_predictions = predictions[:,index:index+1,:,:]
    print(cat_predictions.shape)
    cat_2_predictions_dict[cat] = cat_predictions

person => 0
(30, 1, 224, 224)
vehicle => 1
(30, 1, 224, 224)


In [126]:
def plotPredictedVsReality(pics,previews,predictions):
    
    plt.figure(figsize=(15,15))
    for index in range(len(predictions)):
   
        pic = pics[index]
        preview = previews[index]
        prediction = np.squeeze(predictions[index])        
        
        nr_cols = 3
         
        plt.subplot(1,nr_cols,1)
        plt.imshow(pic)
        
        plt.subplot(1,nr_cols,2)
        plt.imshow(preview)
    
        plt.subplot(1,nr_cols,3)
        plt.imshow(pic)
        plt.imshow(prediction, cmap="cool", alpha=0.5)
       
        plt.figure(figsize=(15,15))
        
    plt.show()

In [136]:
def plotForCategory(cat):
    masks = cat_2_masks[cat][:no_predictions]
    prevs = cat_2_prev[cat][:no_predictions]
    predictions = cat_2_predictions_dict[cat]
    raw_pics = pics[:no_predictions]
    plotPredictedVsReality(raw_pics,prevs,predictions)
    

In [139]:
# plotForCategory('vehicle')

In [140]:
# plotForCategory('person')